# ARIMA

**ARIMA (AutoRegressive Integrated Moving Average)** is a popular statistical method for time series forecasting. ARIMA models are used to understand past data or predict future data points in a series.

## Concept

ARIMA models are composed of three main components:
- **AR (AutoRegressive):** This part uses past values in the time series to predict future values based on a lagged relationship. The AR part involves terms like:
  $$
  \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + \ldots + \phi_p Y_{t-p}
  $$
  where \( Φ_i \) represents the coefficients of the lagged observations.
- **I (Integrated):** This represents the differencing of raw observations to make the time series stationary, i.e., data values are replaced by the difference between the data values and a previous value. The differencing can be represented as:
  $$
  (1 - L)^d Y_t
  $$
  where \( L \) is the lag operator, and \( d \) is the order of differencing.
- **MA (Moving Average):** This involves modeling the error term as a combination of error terms from the past that move with time, expressed as:
  $$
  \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \ldots + \theta_q \epsilon_{t-q}
  $$
  where \( θ_i \) are the coefficients of the moving average terms and \( ε_t \) are the error terms.

## Model Notation

ARIMA models are generally denoted as ARIMA(p, d, q) where:
- **p:** The number of lag observations included in the model (lag order).
- **d:** The number of times that the raw observations are differenced (degree of differencing).
- **q:** The size of the moving average window (order of moving average).

## Stationarity

A key assumption of ARIMA is that the underlying data must be stationary. Stationarity means that the statistical properties such as mean, variance, and autocorrelation are all constant over time. Non-stationary behaviors can be trends, cycles, random walks, or combinations of the three.

## Seasonal ARIMA (SARIMA)

For seasonal effects, ARIMA models can be extended to SARIMA which incorporates seasonal elements into the model. These are denoted as SARIMA(p, d, q)(P, D, Q)[s] where:
- **P, D, Q** represent the seasonal autoregressive order, differencing order, and moving average order, respectively.
- **s:** The number of time steps for a single seasonal period.

## Applications

ARIMA models are widely used in economics, finance, and business for tasks such as:
- Sales forecasting,
- Stock market analysis,
- Economic forecasting.

# Implemetation

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

### Import and show Data

In [2]:
data = pd.read_csv('./Data/BeadArea.csv')
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

,Time,Position Feedback,Velocity Feedback from Axis 1,Velocity Feedback from Axis 2,Temperature,Current Feedback,Interpolated Bead Area
0,0.056,15.567,11.178,6.4952,1.1173,1.2309,1.1184
1,0.057,15.579,11.257,6.4952,1.1175,1.2735,1.1191
2,0.058,15.590,11.375,3.2476,1.1175,1.1567,1.1198
3,0.059,15.601,11.324,6.4952,1.1173,1.2975,1.1205
4,0.060,15.613,11.285,6.4952,1.1173,1.2168,1.1212


The data is composed of  295568  rows and  7  columns.


### Predict Bead Area

In [3]:
window = 100

# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(500, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# ARIMA parameter sliders
p_slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='AR Order (p):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

d_slider = widgets.IntSlider(
    value=1,
    min=0,
    max=2,
    step=1,
    description='Difference Order (d):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

q_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='MA Order (q):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

# Slider to control future prediction steps
future_steps_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Future Steps:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        train_size_pct = train_size_slider.value / 100
        p = p_slider.value
        d = d_slider.value
        q = q_slider.value
        future_steps = future_steps_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data
        y = df['Interpolated Bead Area']
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        y_train, y_test = y[:train_size], y[train_size:train_size+future_steps]
        
        # Fit ARIMA model on training data
        model = ARIMA(y_train, order=(p, d, q))
        model_fit = model.fit()

        # Forecast on the test data and beyond
        forecast_steps = window  # 10 additional future steps
        forecast = model_fit.forecast(steps=forecast_steps)
        
        # Extract predictions for test data
        y_pred = forecast[:future_steps]
        #future_predictions = forecast[len(y_test):]
        
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.axvline(x=y_test.index[-1], color='gray', linestyle='--')  # Mark the end of actual test data
        #future_index = range(y_test.index[-1] + 1, y_test.index[-1] + 1 + len(future_predictions))
        #plt.plot(future_index, future_predictions, label='Future Predictions', color='green', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Interpolated Bead Area')
        plt.title('Actual vs Predicted Interpolated Bead Area')
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test.values - y_pred) ** 2  # Match dimension with y_test
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual Interpolated Bead Area')
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, train_size_slider, p_slider, d_slider, q_slider, future_steps_slider, apply_button, output)


IntRangeSlider(value=(0, 500), continuous_update=False, description='Index Range:', layout=Layout(width='600px…

IntSlider(value=80, continuous_update=False, description='Train %:', layout=Layout(width='600px'), max=95, min…

IntSlider(value=5, continuous_update=False, description='AR Order (p):', layout=Layout(width='600px'), max=10,…

IntSlider(value=1, continuous_update=False, description='Difference Order (d):', layout=Layout(width='600px'),…

IntSlider(value=0, continuous_update=False, description='MA Order (q):', layout=Layout(width='600px'), max=10,…

IntSlider(value=10, continuous_update=False, description='Future Steps:', layout=Layout(width='600px'), style=…

Button(description='Apply Changes', layout=Layout(width='800px'), style=ButtonStyle())

Output()